In [1]:
# hello world
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
import string
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sc = pyspark.SparkContext()
from pyspark.sql.functions import concat_ws, array
from pyspark.sql import *

spark = SparkSession.builder.appName("Projec3")\
        .config("spark.some.config.option","some-value")\
        .getOrCreate()

df = spark.read.load("/home/cse587/CSE587/Project3/train.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')


#punc = string.punctuation
def removePunctuation(s):
    print("type of s: ",s)
    return re.sub(r'[^\w\s]',"",s)
#data = re.sub(r'[^\w\s]',"",data)

udf = UserDefinedFunction(lambda x: removePunctuation(x), StringType())
df = df.withColumn('plot', udf(df.plot))
#new_df = df.withColumn("plot",)

genreMap = spark.read.load("/home/cse587/CSE587/Project3/mapping.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
genreMap = genreMap.withColumnRenamed('_c0','id')
genreMap = genreMap.withColumnRenamed('0','genre')

#this converts a column into a list for use inside encodeGenre() 
#since you cant perform computations on a dataframe inside a UDF
genreList = genreMap.select("genre").rdd.flatMap(lambda x: x).collect()


def encodeGenre(s):
    #s : the value of col "genre" at some row
    retVal=""
    #s = re.sub(r'[^\\'']',"",s)
    #print("s: ",s)
    #print("type of s: ",type(s))
    if(s == None):
        return '00000000000000000000'
    #convert from string to list
    s = s[1:-1].split(", ")
    #print("type of s: ",type(s))
    #print(s)
    
    #remove single quotes, regex wasnt working for some reason
    temp=[]
    for x in s:
        #s.remove(x)
        x=x[1:-1]
#         print(x)
#         index = genreMap["0"==x]
#         print(index)
        temp.append(x)
    s=temp
    #print(s)
    
    #create one hot encoding, 
    for genre in genreList:
        if genre in s:
            retVal=retVal+"1"
        else:
            retVal=retVal+"0"
##### apparetnly it aint kosher to perform df operations inside a udf... so this hour of work was for nothing
#     t = genreMap.withColumn("encoding",F.when(genreMap.genre.isin(s),1).otherwise(0))
#     #t.show()
#     retVal = t.select("encoding").rdd.flatMap(lambda x: x).collect()
#     retVal = ''.join(map(str,retVal))
    return retVal
    
    

udf2 = UserDefinedFunction(lambda x: encodeGenre(x), StringType())
df = df.withColumn('genre',udf2(df.genre))

#install numpy if you havent already, otherwise one of the ml.featuer imports will throw an error cause it needs numpy
#%pip install numpy
import numpy
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

#taken from https://github.com/apache/spark/blob/master/examples/src/main/python/ml/tf_idf_example.py
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
wordsData = tokenizer.transform(df)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)

#this will do part 2
# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)


df_test = spark.read.load("/home/cse587/CSE587/Project3/test.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
df_test = df_test.withColumn('plot',udf(df_test.plot))

tokenizer2 = Tokenizer(inputCol="plot", outputCol="words")
wordsData2 = tokenizer2.transform(df_test)
hashingTF2 = HashingTF(inputCol="words", outputCol="features")
test_cases = hashingTF2.transform(wordsData2)



from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import substring 
from pyspark.sql.types import *

all_label_set = featurizedData.drop('movie_id', 'movie_name', 'plot', 'words')

columns = all_label_set.columns
columns = columns[::-1]
all_label_set = all_label_set[columns]
all_label_set = all_label_set.withColumnRenamed('rawFeatures','features')
all_label_set = all_label_set.withColumnRenamed('genre','all_label')

In [13]:
for i in range(20):
    #substr(start = 0,len)
    x = all_label_set.withColumn("label", all_label_set.all_label.substr(i+1,1).cast(IntegerType()))
    x = x.drop('all_label')
    lr = LogisticRegression(maxIter = 20,regParam=0.0,elasticNetParam=0.0)
    lr_model =lr.fit(x)

    #for all data
    #predict
    print(i)
    test_cases = lr_model.transform(test_cases)
    test_cases = test_cases.withColumn(genreList[i],test_cases['prediction'].cast(IntegerType()))
    test_cases = test_cases.drop('prediction','probability','rawPrediction')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [12]:
#test_cases.filter(test_cases['Drama'] != test_cases['Thriller']).show()
#test_cases = test_cases.withColumn('test', test_cases.movie_id.substr(1,2))
#test_cases.select('movie_id','test').show()
#test_cases = test_cases.drop('test')

SyntaxError: invalid syntax (<ipython-input-12-03b26b674477>, line 2)

In [14]:
test_cases = test_cases.drop('movie_name', 'plot', 'words','features')
test_cases =test_cases.withColumn('all_genre',concat_ws(' ',array([genreList[i] for i in range(20)])))
for i in range(20):
    test_cases = test_cases.drop(genreList[i])

In [60]:
'''
for i in range(20):
    a = test_cases.filter(test_cases[genreList[i]] != 0).filter(test_cases[genreList[i]] != 1)
    a.select('movie_id',genreList[i]).show(vertical=True)
'''
#test_cases.filter(test_cases['movie_id'] < 10).show(vertical=True)
#test_cases.show(vertical=True)

-RECORD 0-------------------------
 movie_id  | 1335380              
 all_genre | 1 1 0 0 0 0 0 0 0... 
-RECORD 1-------------------------
 movie_id  | 29062594             
 all_genre | 1 1 0 0 0 0 0 0 0... 
-RECORD 2-------------------------
 movie_id  | 9252321              
 all_genre | 1 1 0 0 0 0 0 0 0... 
-RECORD 3-------------------------
 movie_id  | 13455076             
 all_genre | 0 0 1 0 0 0 0 0 0... 
-RECORD 4-------------------------
 movie_id  | 24165951             
 all_genre | 0 0 0 0 0 0 0 0 0... 
-RECORD 5-------------------------
 movie_id  | 1925869              
 all_genre | 1 1 0 0 0 0 0 0 0... 
-RECORD 6-------------------------
 movie_id  | 10799612             
 all_genre | 1 1 1 0 0 0 0 0 0... 
-RECORD 7-------------------------
 movie_id  | 28238240             
 all_genre | 0 0 0 0 0 0 0 0 0... 
-RECORD 8-------------------------
 movie_id  | 17124781             
 all_genre | 0 0 0 0 0 1 0 0 0... 
-RECORD 9-------------------------
 movie_id  | 2820794

In [15]:
test_cases.write.option("delimiter", ',').csv('./part1_fixed.csv', escapeQuotes=False)